In [1]:
import os
import sys
import pencil as pc
import matplotlib
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import modes
import pencil as pc

         pencil has moved to pencil_old.
To change your scripts accordingly:
import pencilnew as pc -> import pencil as pc
import pencil as pc -> import pencil_old as pc


In [2]:
# matplotlib.use('Agg')  # NOTE: changing the matplotlib backend

# plt.rcParams.update({'font.size': 14})
# plt.rcParams['text.usetex'] = True

class Butterfly(modes.Modes):
    def __init__(self, path, sim_path, sim, ts, xyaver, yaver, ini=False, dyn=True):
        self.dyn = dyn
        super().__init__(sim, ts, xyaver, yaver, dyn=self.dyn)#, ini=False)

        self.path = path
        self.by = [self.bxmz, self.bymz]
        # self.by2 = [self.bx2mz, self.by2mz]
        # self.by2sqr = np.sqrt(self.by2)
        try:
            self.by2 = [self.bx2mz, self.by2mz]
            self.by2sqr = np.sqrt(self.by2)
        except AttributeError:
            print("Have you included bx2mz and by2mz in xyaver.in?")
            print("Diagram will not be normalized!!!")


    def plot(self):
        """
        function to plot butterfly diagram.
        """
        
        left, width = .488, .5
        bottom, height = .45, .5
        right = left + width
        top = bottom + height

        gs_kw = dict(width_ratios=[2, 0.1], height_ratios=[1, 1])
        fig, axs = plt.subplot_mosaic([['upper left', 'right'],
                                    ['lower left', 'right']],
                                    gridspec_kw=gs_kw, figsize=(8,5),
                                    layout="constrained")

        props = dict(boxstyle='round', facecolor='white', alpha=0.8)

        pos = ['upper left', 'lower left']

        [X, Y] = np.meshgrid(self.xyaver.t, self.z)

        try:
            vmax=np.max(self.by/self.by2sqr)
            vmin=np.min(self.by/self.by2sqr)
            title = [r'$\langle B_x \rangle _{xy}/\sqrt{\langle B_{\vphantom{y}x}^2 \rangle _{xy}}$', r'$\langle B_y \rangle _{xy}/\sqrt{\langle B_y^2 \rangle _{xy}}$']
            for i in range(len(pos)):
                axs[pos[i]].contourf(X, Y, np.transpose(self.by[i]/self.by2sqr[i]), 100, cmap='bwr', vmin=vmin, vmax=vmax)
                # axs[pos[i]].set_title(title[i])
                axs[pos[i]].text(right, top, title[i], transform=axs[pos[i]].transAxes, fontsize=12, horizontalalignment='right', verticalalignment='top', bbox=props)
                axs[pos[i]].set_xlabel(r'$t$')
                axs[pos[i]].set_ylabel(r'$z$')

            ims = axs[pos[0]].contourf(X, Y, np.transpose(self.by[0]/self.by2sqr[0]), 100, cmap='bwr', vmin=vmin, vmax=vmax)

        except Exception:
            vmax=np.max(self.by)
            vmin=np.min(self.by)
            title = [r'$\langle B_x \rangle _{xy}$', r'$\langle B_y \rangle _{xy}$']
            for i in range(len(pos)):
                axs[pos[i]].contourf(X, Y, np.transpose(self.by[i]), 100, cmap='bwr', vmin=vmin, vmax=vmax)
                # axs[pos[i]].set_title(title[i])
                axs[pos[i]].text(right, top, title[i], transform=axs[pos[i]].transAxes, fontsize=12, horizontalalignment='right', verticalalignment='top', bbox=props)
                axs[pos[i]].set_xlabel(r'$t$')
                axs[pos[i]].set_ylabel(r'$z$')

            ims = axs[pos[0]].contourf(X, Y, np.transpose(self.by[0]), 100, cmap='bwr', vmin=vmin, vmax=vmax)

        plt.tight_layout()
        plt.suptitle(r'$Butterfly$ $Diagram$')
        plt.subplots_adjust(top=0.92)
        cbar = fig.colorbar(ims, cax=axs['right'])#, ticks=[-.3, -.2, -.1, 0, .1, .2, .3])
        ticks = np.arange(round(vmin,2), round(vmax,2), 0.1)
        cbar.set_ticks(ticks)
        fig.tight_layout()
        # plt.savefig(self.path+'butterfly.pdf',bbox_inches='tight',dpi=300)
        plt.savefig(self.path+'butterfly.png',bbox_inches='tight',dpi=300)
        # plt.show()

    pass

: 

In [3]:
#!! location of modules
modulePath = './scripts'
sys.path.append(modulePath)

#!! importing my modules
# import statevariables
# import rms
import butterfly
import kom

plt.rcParams.update({'font.size': 14})
# plt.rcParams['text.usetex'] = True


#!! location of working directory
script_path = os.getcwd()
sim_path = os.path.abspath(os.path.join(script_path))#, os.pardir))
print('Simulation directory: ', sim_path)
data_dir = '/data'


print('importing data ...')
# reading the simulation object and other files
sim = pc.get_sim(path=sim_path, quiet=True)
ts = pc.read.ts(datadir=sim_path+data_dir, quiet=True)
yaver = pc.read.aver(datadir=sim_path+data_dir, simdir=sim_path, plane_list='y')
xyaver = pc.read.aver(datadir=sim_path+data_dir, simdir=sim_path, plane_list=['xy'])

In [ ]:
pathexists = os.path.exists(path)
if pathexists:
    print(path + ' exists.')
else:
    print(path + ' does not exist.')
    os.makedirs(path)
    print(path + ' is created.')
plot_dir = os.path.abspath(sim_path+'/plot')
print('Output directory: ', plot_dir)

In [ ]:
# creating an Butterfly object
img_Bfly = butterfly.Butterfly(path, sim_path, sim, ts, xyaver, yaver, ini=False, dyn=True)
# plotting butterfly diagram
img_Bfly.plot()